# snow flake 계정을 주피터 노트북에 연결

In [2]:
from dotenv import load_dotenv
import os
import snowflake.connector
# .env 파일의 환경변수를 로드

load_dotenv(dotenv_path='dev.env')


True

In [3]:
from dotenv import load_dotenv
import os
import snowflake.connector

# .env 파일의 환경변수를 로드
load_dotenv(dotenv_path='dev.env')

def return_snowflake_conn():

    conn = snowflake.connector.connect(
        user=os.environ.get('SNOWFLAKE_USER'),
        password=os.environ.get('SNOWFLAKE_PASSWORD'),
        account=os.environ.get('SNOWFLAKE_ACCOUNT'),
        warehouse = os.environ.get('SNOWFLAKE_WAREHOUSE'),
        database = os.environ.get('SNOWFLAKE_DATABASE')
    )

    return conn.cursor()

In [8]:
import requests

def extract(url):
    f = requests.get(url)
    return (f.text)

In [9]:
def transform(text):
    lines = text.strip().split("\n")
    records = []
    for l in lines:
      (country, capital) = l.split(",")
      records.append([country, capital])

    return records

In [10]:
def load(cur, records):
    # full refresh로 구현
    # 먼저 테이블을 생성
    target_table = "dev.raw_data.country_capital"
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {target_table} (
      country varchar primary key,
      capital varchar
    )""")
    # 레코드를 적재
    for r in records:
        # '와 같은 특수 문자가 INSERT INTO를 깨는 것을 막기 위해 '를 ''로 변경
        # 이 대신에 parameterized INSERT INTO를 대신 사용해도 됨
        country = r[0].replace("'", "''")
        capital = r[1].replace("'", "''")
        sql = f"INSERT INTO {target_table} (country, capital) VALUES ('{country}', '{capital}')"
        print(country, "-", capital, "-", sql)
        cur.execute(sql)

In [11]:
link = "https://s3-geospatial.s3.us-west-2.amazonaws.com/country_capital.csv"

data = extract(link)

In [12]:
len(data)

5069

In [13]:
lines = transform(data)

In [14]:
len(lines)

248

In [15]:
lines[0:10]

[['country', 'capital'],
 ['Abkhazia', 'Sukhumi'],
 ['Afghanistan', 'Kabul'],
 ['Akrotiri and Dhekelia', 'Episkopi Cantonment'],
 ['Albania', 'Tirana'],
 ['Algeria', 'Algiers'],
 ['American Samoa', 'Pago Pago'],
 ['Andorra', 'Andorra la Vella'],
 ['Angola', 'Luanda'],
 ['Anguilla', 'The Valley']]

In [18]:
cur = return_snowflake_conn()
load(cur, lines)

country - capital - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('country', 'capital')
Abkhazia - Sukhumi - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Abkhazia', 'Sukhumi')
Afghanistan - Kabul - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Afghanistan', 'Kabul')
Akrotiri and Dhekelia - Episkopi Cantonment - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Akrotiri and Dhekelia', 'Episkopi Cantonment')
Albania - Tirana - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Albania', 'Tirana')
Algeria - Algiers - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Algeria', 'Algiers')
American Samoa - Pago Pago - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('American Samoa', 'Pago Pago')
Andorra - Andorra la Vella - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Andorra', 'Andorra la Vella')
Angola - Luanda - INSERT INT

In [19]:
# table를 읽어서 판다스 데이터프레임으로 변환하고 처음 5개의 레코드와 레코드 수를 표시
def check_table_stats(cur, table, key):
    result = cur.execute(f"SELECT * FROM {table} ORDER BY {key}")
    df = cur.fetch_pandas_all()
    print(len(df))
    print(df.head())
    return df

In [20]:
check_table_stats(cur, "dev.raw_data.country_capital", "country")

744
       COUNTRY  CAPITAL
0     Abkhazia  Sukhumi
1     Abkhazia  Sukhumi
2     Abkhazia  Sukhumi
3  Afghanistan    Kabul
4  Afghanistan    Kabul


,COUNTRY,CAPITAL
0,Abkhazia,Sukhumi
1,Abkhazia,Sukhumi
2,Abkhazia,Sukhumi
3,Afghanistan,Kabul
4,Afghanistan,Kabul
...,...,...
739,Zimbabwe,Harare
740,Zimbabwe,Harare
741,country,capital
742,country,capital


## ETL 코드를 다시 한번 실행하고 멱등성이 지켜지는지 확인

In [21]:
data = extract(link)
lines = transform(data)
cur = return_snowflake_conn()
load(cur, lines)

country - capital - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('country', 'capital')
Abkhazia - Sukhumi - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Abkhazia', 'Sukhumi')
Afghanistan - Kabul - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Afghanistan', 'Kabul')
Akrotiri and Dhekelia - Episkopi Cantonment - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Akrotiri and Dhekelia', 'Episkopi Cantonment')
Albania - Tirana - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Albania', 'Tirana')
Algeria - Algiers - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Algeria', 'Algiers')
American Samoa - Pago Pago - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('American Samoa', 'Pago Pago')
Andorra - Andorra la Vella - INSERT INTO dev.raw_data.country_capital (country, capital) VALUES ('Andorra', 'Andorra la Vella')
Angola - Luanda - INSERT INT

In [23]:
# 결과를 보면 Primary Key Uniqueness가 지켜지지 않음을 알 수 있다
check_table_stats(cur, "dev.raw_data.country_capital", "country")

992
       COUNTRY  CAPITAL
0     Abkhazia  Sukhumi
1     Abkhazia  Sukhumi
2     Abkhazia  Sukhumi
3     Abkhazia  Sukhumi
4  Afghanistan    Kabul


,COUNTRY,CAPITAL
0,Abkhazia,Sukhumi
1,Abkhazia,Sukhumi
2,Abkhazia,Sukhumi
3,Abkhazia,Sukhumi
4,Afghanistan,Kabul
...,...,...
987,Zimbabwe,Harare
988,country,capital
989,country,capital
990,country,capital
